In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time
import undetected_chromedriver as uc
from io import StringIO
from bs4 import BeautifulSoup
import pandas as pd

import os

# HOLERITE

### config

In [14]:

# mas geralmente funciona assim:
driver = uc.Chrome()

driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)



In [4]:
# LINKS

lista_senadores = [
    {'Nome': 'JORGE SEIF', 'Link': 'https://www6g.senado.leg.br/transparencia/sen/6342/?ano=2025'},
    {'Nome': 'AMIN',       'Link': 'https://www6g.senado.leg.br/transparencia/sen/22/?ano=2025'},
    {'Nome': 'IVETE',      'Link': 'https://www6g.senado.leg.br/transparencia/sen/6010/?ano=2025'}
]

df_senadores = pd.DataFrame(lista_senadores)

In [5]:
driver.get(df_senadores['Link'][0])

### Extracao Senador

In [5]:
def extrair_dados_holerite(html_content, referencia):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 1. Extração dos Dados Cadastrais (Cabeçalho)
    # Ficam na div com class="span3"
    dados_cadastrais = {}
    div_topo = soup.find('div', class_='span3')
    
    if div_topo:
        # Pega cada div filha dentro do cabeçalho (ex: Nome, Vínculo, etc)
        for item in div_topo.find_all('div'):
            texto = item.get_text(strip=True)
            if ':' in texto:
                dados_cadastrais['Referencia'] = referencia
                chave, valor = texto.split(':', 1)
                dados_cadastrais[chave.strip()] = valor.strip()
    
    # Lista para guardar todas as folhas encontradas (Normal, Suplementar, etc)
    registros_pagamento = []
    
    # 2. Extração das Tabelas de Remuneração
    # O segredo: find_all pega TODAS as tabelas, não importa quantas existam
    tabelas = soup.find_all('table', class_='tabela_resposta')
    
    for tabela in tabelas:
        # Começamos um dicionário novo copiando os dados cadastrais (Nome, etc)
        dados_folha = dados_cadastrais.copy()
        
        # Iteramos sobre todas as linhas (tr) da tabela
        for linha in tabela.find_all('tr'):
            colunas = linha.find_all('td')
            
            # Só nos interessa linhas que têm 2 colunas (Chave e Valor)
            if len(colunas) == 2:
                chave = colunas[0].get_text(strip=True)
                valor = colunas[1].get_text(strip=True)
                
                # Ignora linhas vazias ou separadores
                if chave and valor:
                    dados_folha[chave] = valor
        
        # Adiciona essa folha processada à lista
        registros_pagamento.append(dados_folha)
        
    # 3. Extração de Informações Suplementares (Retroativos)
    # O arquivo 2 tem uma div específica para isso fora da tabela
    box_suplementar = soup.find('div', class_='box_suplementar2')
    if box_suplementar:
        texto_sup = box_suplementar.get_text(" ", strip=True)
        # Se houver texto, podemos adicionar ao último registro ou criar um campo de obs
        if registros_pagamento:
             registros_pagamento[-1]['Observacoes_Suplementares'] = texto_sup

    return registros_pagamento

In [81]:
def captcha():
    driver.switch_to.frame(driver.find_element(By.CSS_SELECTOR, 'iframe[title="reCAPTCHA"]')) # Muda para iframe selecionado com find_element
    driver.find_element(By.CSS_SELECTOR, '#recaptcha-anchor').click()
    driver.switch_to.default_content() # Sai do iframe e volta para o HTML principal

In [37]:
def dict_to_df(dict_):
    df = pd.DataFrame(dict_)
    return df

In [35]:
registros = []
for index, linha in df_senadores.iterrows():
    nome_senador = linha['Nome']
    link_senador = linha['Link']
    
    print(f'Acessando dados do {nome_senador}')
    print(f'Link do portal {link_senador}')
    # Acessando link
    driver.get(link_senador)
    
    # Clicando em consulte (Contrachques Senador)
    driver.find_element(By.CSS_SELECTOR, '#collapse-subsidios a').click()
    
    time.sleep(15) # Esperando captcha
    
    for i in range(0,12): # De dezembro à Janeiro, pode ser que nao funcione daqui um mes alterar se for o caso
    # Select para mes a mes 
        select = driver.find_element(By.CSS_SELECTOR, 'select').get_property('children') # Para ter o log de mes
        data = Select(driver.find_element(By.CSS_SELECTOR, 'select'))
        data.select_by_index(i)
        
        data_selecionado = select[i].text
        print(f'Selecionando mês: {data_selecionado}', end='\r')
        
        driver.find_element(By.CSS_SELECTOR, 'input[type="submit"]').click()
        
        time.sleep(1.5) 
        html = driver.page_source
        registro = extrair_dados_holerite(html, data_selecionado)
        registros.extend(registro) # Usa foramto extend para salvar na lista
        
        driver.back()
        

Acessando dados do JORGE SEIF
Link do portal https://www6g.senado.leg.br/transparencia/sen/6342/?ano=2025
Acessando dados do AMIN25
Link do portal https://www6g.senado.leg.br/transparencia/sen/22/?ano=2025
Acessando dados do IVETE5
Link do portal https://www6g.senado.leg.br/transparencia/sen/6010/?ano=2025


In [ ]:
holerite_senadores = dict_to_df(registros)
holerite_senadores.to_csv('Holerite_senadores.csv')

### Extacao pessoal dos senadores

Pegando links para extrair holerites

In [ ]:
lista = []
for index, linha in df_senadores.iterrows():
    
    link_senador = linha['Link']
    nome_senador = linha['Nome']
    driver.get(link_senador)
    time.sleep(0)
    linhas = driver.find_elements(By.CSS_SELECTOR, '#collapse-pessoal > div > table > tbody tr')
        
    for linha in linhas:
        if linha.get_attribute('class') == 'sen_tabela_linha_grupo':
            children = linha.get_property('children')
            local = children[0].text
        else:
            children = linha.get_property('children')
            vinculo = children[0].text
            link = children[1].find_element(By.TAG_NAME, 'a').get_attribute('href')
            dicionario = {
                'Senador':nome_senador,
                'local':local,
                'vinculo':vinculo,
                'link':link
                }
            lista.append(dicionario)
            
df_link_cargos = pd.DataFrame(lista)

Dos links cargos, aqui pego os links dos funcionarios

In [ ]:
registros = []
for index, linha in df_link_cargos.iterrows():
    senador = linha['Senador']
    local = linha['local']
    vinculo = linha['vinculo']
    link = linha['link']
    print(vinculo)
    print(f'Acessando dados do {senador}, do {local} {vinculo}', end='\r')
    driver.get(link)
    if vinculo.strip() == 'Comissionados':
        lines = driver.find_elements(By.CSS_SELECTOR, '#comissionados > table > tbody > tr')
    elif vinculo.strip() == 'Efetivos':
        lines = driver.find_elements(By.CSS_SELECTOR, '#efetivos > table > tbody > tr')
    elif vinculo.strip() == 'Terceirizados':
        continue
    
    for line in lines:
        child = line.get_property('children')

        registro = {
            'senador':senador,
            'local':local,
            'vinculo':vinculo,
            'nome_func':child[0].text,
            'funcao':child[1].text,
            'nome_funcao':child[2].text,
            'link_funcionario': child[0].find_element(By.TAG_NAME, 'a').get_attribute('href'),
        }
        registros.append(registro)
    
    time.sleep(1.5) 
    
df_funcionarios = pd.DataFrame(registros)

In [148]:
df_funcionarios.to_excel('funcionarios_senadores.xlsx',index=False)

Extracao holerite

In [162]:
import capsolver

In [163]:
capsolver.api_key = "CAP-F8A5FA171D4DB772546BB20EB38A5208F872E76A8037B4F4224453C2FFECC63F"
# Coloque a SiteKey que você pegou no HTML do Senado
SITE_KEY_SENADO = "6Lecp4saAAAAAGyhwly32P0V6knSxuKJI6YhVlvb"

def resolver_captcha_senado(driver, url_atual):
    """
    Tenta resolver o captcha 3 vezes. 
    Retorna True se conseguir, False se falhar todas.
    """
    max_tentativas = 3
    
    for tentativa in range(1, max_tentativas + 1):
        try:
            print(f"🧩 Tentativa de resolver Captcha {tentativa}/{max_tentativas}...")
            
            # 1. Solicita a resolução ao Capsolver
            solution = capsolver.solve({
                "type": "ReCaptchaV2TaskProxyLess",
                "websiteURL": url_atual,
                "websiteKey": SITE_KEY_SENADO
            })
            
            token = solution['gRecaptchaResponse']
            
            # 2. Injeta o token mágico na página (simula a resolução)
            # Isso preenche o campo invisível que o servidor valida
            driver.execute_script(f"""
                document.getElementById('g-recaptcha-response').innerHTML = '{token}';
                document.getElementById('g-recaptcha-response').value = '{token}';
            """)
            
            print(f"✅ Captcha resolvido e injetado com sucesso!")
            return True

        except Exception as e:
            print(f"❌ Erro na tentativa {tentativa}: {e}")
            if tentativa < max_tentativas:
                print("⏳ Aguardando 2 segundos antes de tentar de novo...")
                time.sleep(2)
            else:
                print("💀 Falha crítica: Não foi possível resolver o captcha após 3 tentativas.")
                return False

In [169]:
def esperar_cap():
    print("Aguardando a extensão resolver o Captcha...")

    # O segredo: Esperamos até que o input escondido 'g-recaptcha-response' tenha algum valor
    WebDriverWait(driver, 120).until(
        lambda d: d.find_element(By.ID, "g-recaptcha-response").get_attribute("value") != ""
    )

    print("Captcha resolvido! Prosseguindo...")

In [172]:
registros = []

for index, linha in df_funcionarios.iterrows():
    nome_func = linha['nome_func']
    senador = linha['senador']
    local = linha['local']
    link_func = linha['link_funcionario']
    
    print(f'---------------------------------------------------')
    print(f'Acessando: {nome_func} - Senador {senador}')
    
    driver.get(link_func)
    
    # CHAMADA DA FUNÇÃO AQUI
    # Se retornar False (não conseguiu), pulamos para o próximo funcionário
    if not resolver_captcha_senado(driver, link_func):
        print(f"Pulando {nome_func} por falha no captcha.")
        continue 
    
    # Se chegou aqui, o captcha está resolvido. O token vale por 2 min.
    
    try:
        # Loop dos meses
        for i in range(0, 12): 
            # Re-encontrar o elemento a cada iteração evita erro de "stale element"
            select_element = driver.find_element(By.CSS_SELECTOR, 'select')
            opcoes = select_element.get_property('children')
            
            # Seleciona o mês
            data = Select(select_element)
            data.select_by_index(i)
            
            data_selecionado = opcoes[i].text
            print(f'\t📅 Processando mês: {data_selecionado}', end='\r')
            
            # Clica no botão de visualizar
            driver.find_element(By.CSS_SELECTOR, 'input[type="submit"]').click()

            # Espera a tabela carregar (melhor que time.sleep fixo)
            # Ajuste o 'time.sleep' se a tabela demorar a renderizar o HTML
            time.sleep(1.5) 
            
            html = driver.page_source
            registro = extrair_dados_holerite(html, data_selecionado)

            for r in registro:
                r['senador'] = senador
                r['local'] = local
            registros.extend(registro)
            
            # Volta para a página anterior (onde o captcha já está "resolvido")
            driver.back()
            
            # Pequena pausa para garantir que o navegador processou a volta
            time.sleep(0.5)

    except Exception as e:
        print(f"\nErro ao processar meses de {nome_func}: {e}")
        continue

print("\nColeta finalizada!")

---------------------------------------------------
Acessando: ADNA DOS ANJOS CAJUEIRO - Senador JORGE SEIF
🧩 Tentativa de resolver Captcha 1/3...
✅ Captcha resolvido e injetado com sucesso!
---------------------------------------------------
Acessando: ANDRÉIA LINS RIBAS - Senador JORGE SEIF
🧩 Tentativa de resolver Captcha 1/3...
✅ Captcha resolvido e injetado com sucesso!
---------------------------------------------------
Acessando: MICHELLI REGLY ALVES BORTOLOTTI - Senador JORGE SEIF
🧩 Tentativa de resolver Captcha 1/3...
✅ Captcha resolvido e injetado com sucesso!
---------------------------------------------------
Acessando: RAFAEL AUGUSTO LUISI DE OLIVEIRA - Senador JORGE SEIF
🧩 Tentativa de resolver Captcha 1/3...
✅ Captcha resolvido e injetado com sucesso!
---------------------------------------------------
Acessando: CAROLINE EIDT DUARTE - Senador JORGE SEIF
🧩 Tentativa de resolver Captcha 1/3...
✅ Captcha resolvido e injetado com sucesso!
------------------------------------

In [173]:
df_holerite_func = pd.DataFrame(registros)
df_holerite_func.to_csv('holerite_pessoal.csv', index=False)

In [17]:
viagens = []
for i, row in df_senadores.iterrows():
    nome = row['Nome']
    link = row['Link']
    print(link)
    driver.get(link)
    valor = driver.find_element(By.CSS_SELECTOR, '#collapse-outros-gastos > div > table > tbody > tr:nth-child(2) > td.valor > a').text
    
    viagem_oficial = {
        'nome': nome,
        'valor':valor
    }
    viagens.append(viagem_oficial)

https://www6g.senado.leg.br/transparencia/sen/6342/?ano=2025
https://www6g.senado.leg.br/transparencia/sen/22/?ano=2025
https://www6g.senado.leg.br/transparencia/sen/6010/?ano=2025


In [20]:
pd.DataFrame(viagens).to_csv('Viagens Oficiais.csv', index=False)